# Imports

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns

import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout 
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import classification_report,confusion_matrix

import tensorflow as tf

import cv2
import os

import numpy as np
import copy

# Get Data

In [2]:
img_size_x = 680 #FIXME
img_size_y = 454 #FIXME

def get_data(data_dir):
    data = [] 
    try:
        img_arr = cv2.imread(data_dir) #convert BGR to RGB format, since imread returns BGR
        resized_arr = cv2.resize(img_arr, (img_size_x, img_size_y))[...,::-1] # Reshaping images to preferred size
        data = resized_arr
    except Exception as e:
        print(e)
    return np.array(data)

In [3]:
background = get_data('./Images/location/background2.jpg')
image = get_data('./Images/location/image2.jpg')
#background
#print(background[3,2])

### Display Images

In [4]:
cv2.imshow("image", background[...,::-1])
cv2.waitKey()

-1

In [5]:
cv2.imshow("image", image[...,::-1])
cv2.waitKey()

-1

# Delta-e metrix  

#### Rank system (0-100)
<= 1.0: Not perceptible by the human eye  
1-2: Perceptible through close observation  
2-10: Perceptible at a glance  
11-49: Colors are more similar than the opposite  
100: Colors are exactly the opposite  

http://zschuessler.github.io/DeltaE/learn/

In [7]:
from colormath.color_objects import sRGBColor, LabColor
from colormath.color_conversions import convert_color
from colormath.color_diff import delta_e_cie2000

In [16]:
data = np.zeros(background.shape)
#delta_e_data = np.zeros(background.shape)
for row in range(background.shape[0]): #row
    for col in range(background.shape[1]): #col
        #~4s to convert to rgb
        color1_rgb = sRGBColor(background[row,col,0], background[row,col,1], background[row,col,2], is_upscaled=True);
        color2_rgb = sRGBColor(image[row,col,0], image[row,col,1], image[row,col,2], is_upscaled=True);
        #~24s to convert to labcolor
        color1_lab = convert_color(color1_rgb, LabColor);
        color2_lab = convert_color(color2_rgb, LabColor);
        #~50s to figure out delta_e
        delta_e = delta_e_cie2000(color1_lab, color2_lab); 
        #delta_e_data[row, col] = [delta_e, delta_e, delta_e]
        if (delta_e <= 30):
            data[row,col] = [255,255,255] #white for similar
        elif (delta_e > 30):
            data[row,col] = [0,0,0] #black for different
data = data.astype(np.uint8)

### Options
1. Color comparison, using delta_E (~80s)
2. Low noise, use a white board and convert anything that isn't white to black (theoretically much faster)

In [9]:
cv2.imshow("image", data)
cv2.waitKey()

-1

# Find Center of Mass

In [10]:
from PIL import Image
import numpy as np
import math

immat = copy.deepcopy(data)
(X,Y) = (background.shape[0], background.shape[1])
m = np.zeros((X, Y))
'''
for x in range(X):
    for y in range(Y):
        if immat[x,y,0] != 255 and immat[x,y,1] != 255 and immat[x,y,2] != 255:
            m[x,y] = 0;
        else:
            m[x,y] = 1; 
'''
m = np.sum(immat, -1) < 255*3
m = m / np.sum(np.sum(m))

# marginal distributions
dx = np.sum(m, 1)
dy = np.sum(m, 0)

# expected values
cx = np.sum(dx * np.arange(X))
cy = np.sum(dy * np.arange(Y))

data[math.ceil(cx)][math.ceil(cy)] = [255,0,0]
print(cx,cy)

223.60311080041942 514.2543254106959


-1

In [ ]:
cv2.imshow("image", data[...,::-1]) #red dot in the center of mass
cv2.waitKey()

### Determining location from pixel number
1. If we know much distance a pixel covers, we can figure out where the object is
2. Will require some extra code, or use (0,0) as the bottom left relative to the camera
3. Draw a black line measuring 1 cm, and use code to determine how many pixels that is (run ONCE)